Credit where credit is due: this is based on  https://www.kaggle.com/code/gpreda/fast-test-of-llama-v2-pre-quantized-with-llama-cpp.

Loading the model will take a while so please be patient

In [1]:
# clean up after failed efforts if necessary
!rm -rf llama.cpp
# we need no CUDA for CPU; this flag has changed since the initial release
!CMAKE_ARGS="-DLLAMA_CUDA=off" pip install llama-cpp-python 
!git clone https://github.com/ggerganov/llama.cpp.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 20.6 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - \ | done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.78-cp310-cp310-linux_x86_64.whl size=3658472 sha256=f6874150e6b20b15ce0a29a4bdf3671f27f62f9c21769fbbcda606e645492ecf
  Stored in directory: /root/.cache/pip/wheels/fd/c5/bd/3b1c20081bd71ce9d28b562573c97915c790bf1ef231879a61
Successfully built llama-cpp-python
Cloning into 'llama.cpp'...
remote: Enumerating objects: 27254, done.
remote: Counting objects: 100% (5457/5457), done.
remote: Compressing objects: 100% (419/419), done.
remote: Total 27254 (delta 5183), reused 5174 (delta 5036), pack-reused 21797
Receiving objects: 100% (27254/2

In [2]:
import arrow
from llama_cpp import Llama

time_load = arrow.now()
llm = Llama(model_path="/kaggle/input/test-llama-2-quantized-with-llama-cpp/llama-7b.gguf")
print("Model load time: {:5.4f} seconds".format((arrow.now() - time_load).total_seconds()))

llama_model_loader: loaded meta data with 18 key-value pairs and 291 tensors from /kaggle/input/test-llama-2-quantized-with-llama-cpp/llama-7b.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = 7b-chat-hf
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.

Model load time: 47.0506 seconds


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '2048', 'general.name': '7b-chat-hf', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '32', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'tokenizer.ggml.model': 'llama', 'general.file_type': '7'}
Using fallback chat format: llama-2


In [3]:
def ask(query: str, max_tokens: int):
    time_ask = arrow.now()
    output = llm("Q: {} A: ".format(query), max_tokens=max_tokens, stop=["Q:", "\n"], echo=False)
    return [output['choices'][index]['text'] for index in range(len(output['choices']))], (arrow.now() - time_ask).total_seconds()

query_cities = 'What are the names of three European capital cities?'
result_cities, time_cities = ask(query=query_cities, max_tokens=40)
print(query_cities)
print(result_cities)
print('time: {:5.4f}'.format(time_cities))


llama_print_timings:        load time =    3339.59 ms
llama_print_timings:      sample time =      17.02 ms /    21 runs   (    0.81 ms per token,  1234.21 tokens per second)
llama_print_timings: prompt eval time =    3339.09 ms /    16 tokens (  208.69 ms per token,     4.79 tokens per second)
llama_print_timings:        eval time =   13899.35 ms /    20 runs   (  694.97 ms per token,     1.44 tokens per second)
llama_print_timings:       total time =   17287.69 ms /    36 tokens


What are the names of three European capital cities?
['1. - London, England 2. - Paris, France 3. - Rome, Italy']
time: 17.2992


In [4]:
query_catcher = 'What can you tell me about the J. D. Salinger novel The Catcher in the Rye?'
result_catcher, time_catcher = ask(query=query_catcher, max_tokens=120)
print(query_catcher)
print(result_catcher)
print('time: {:5.4f}'.format(time_catcher))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3339.59 ms
llama_print_timings:      sample time =      76.52 ms /   120 runs   (    0.64 ms per token,  1568.32 tokens per second)
llama_print_timings: prompt eval time =    4241.26 ms /    25 tokens (  169.65 ms per token,     5.89 tokens per second)
llama_print_timings:        eval time =   45260.10 ms /   119 runs   (  380.34 ms per token,     2.63 tokens per second)
llama_print_timings:       total time =   49697.16 ms /   144 tokens


What can you tell me about the J. D. Salinger novel The Catcher in the Rye?
[" Sure, I'd be happy to help! The Catcher in the Rye is a classic novel by J.D. Salinger that was first published in 1951. It tells the story of Holden Caulfield, a disillusioned teenager who runs away from his boarding school and spends three days wandering around New York City. The novel explores themes of alienation, rebellion, and the struggle to find one's place in the world. It is known for its candid portrayal of adolescent"]
time: 49.7122
